In [1]:
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import math
import random
import numpy as np
import os
import pandas as pd
from copy import copy
from Cindy_stimuli import *
from scipy.misc import imread
import pdb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
a = PopoutGratings()
a.create()

600


In [4]:
b = PopoutCircles()
b.create()

600


1

In [ ]:
# a = pd.DataFrame

In [ ]:
# a = imread('/om/user/xf15/NN/project/test_popout_green_circles_on_grid/cat09image0000.png')
# a.shape

In [ ]:
# dfImages
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    if xB - xA < 0: 
        return 0
    if yB - yA < 0:
        return 0

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
conv1_feat = np.load('a.npy')

In [ ]:
conv1_feat.shape

In [ ]:
single_item_width = 80

In [ ]:
# my_rois = np.load('clutter_rois.npy') 
# my_rois = np.load('my_rois.npy')
# my_rois = np.load('red_popout_rois.npy') 
my_rois = np.load('red_popout_rois_1.npy')
# my_rois = np.load('grating_rois_1.npy')
# my_rois = np.load('popout_rois.npy') 
my_rois = my_rois[0,:,:]
# image = Image.new("RGB", (600, 800))
image = Image.open('/om/user/xf15/NN/project/test_popout_green_circles_on_grid/cat09image0000.png')
# image = Image.open('/om/user/xf15/NN/project/test_colorful_circles_on_grid/cat09image0001.png')
# image = Image.open('/om/user/xf15/NN/project/test_colorful_circles/cat10image0000.png')
num_box = my_rois.shape[0] #300
min_rgb = 20
max_rgb = 256
# rs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)
# gs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)
# bs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)

# gs = np.repeat(0, num_box)
# bs = np.repeat(0, num_box)
small_rois = np.ndarray((1,4), dtype="float32")
for iB in range(num_box-1):
    if my_rois[iB, 3] - my_rois[iB, 1] < single_item_width and my_rois[iB, 3] - my_rois[iB, 1] < 80:
        small_rois = np.vstack((small_rois, my_rois[iB, 1:5]))
small_rois = small_rois[1:,:]        
num_small_rois = small_rois.shape[0] #56
rs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))
gs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))
bs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))
# print(my_rois.shape)
# print(small_rois.shape)


In [ ]:
def bb_iou_multi(boxes1, boxes2):
    x11, y11, x12, y12 = np.split(boxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(boxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA), 0) * np.maximum((yB - yA), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)

    return iou


In [ ]:
def bb_iou_single(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = np.maximum((xB - xA), 0) * np.maximum((yB - yA), 0)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / (boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# b = np_vec_no_jit_iou(my_rois, my_rois)
# bb = bb_intersection_over_union(my_rois[1, :], my_rois[298, :])
b = bb_iou_multi(small_rois, small_rois)
bb = bb_iou_single(small_rois[1, :], small_rois[89, :])
print(b[1][89])
print(bb)

In [ ]:
# b>0
my_rois.shape
print(b.shape[1])
b
np.ndarray((1, 4), dtype="float32")

In [ ]:
c = b.sum(axis = 0)
d = np.where(c < 2)[0] 
# d = np.where(c < np.mean(c) - 2 * np.std(c))[0] # use mean and std

# print(b.shape)
# print(c.shape)
# print(d.shape)

print(np.mean(c))
print(np.std(c))

# import matplotlib.pyplot as plt
# plt.imshow(b);
# plt.colorbar()
# plt.show()

In [ ]:
image0 = copy(image)
draw0 = ImageDraw.Draw(image0)
for iB in range(num_small_rois-1):
#     draw1.rectangle(small_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
    draw0.rectangle(my_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
# image1.show()
# image1.save('temp.png')
image0

In [ ]:
image0 = copy(image)
draw1 = ImageDraw.Draw(image0)
for iB in range(num_small_rois-1):
    draw1.rectangle(small_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
#     draw1.rectangle(my_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
# image1.show()
# image1.save('temp.png')
image0

In [ ]:
image1 = copy(image)
draw1 = ImageDraw.Draw(image1)
for iB in range(num_small_rois-1):
    draw1.rectangle(small_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
#     draw1.rectangle(my_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
# image1.show()
# image1.save('temp.png')
image1

In [ ]:
image2 = copy(image)
draw2 = ImageDraw.Draw(image2)
# for iB in range(num_small_rois-1):
# for iB in d:
for iB in list(set(range(num_small_rois-1)) - set(d)):
    draw2.rectangle(small_rois[iB, :], outline = (rs[iB], gs[iB], bs[iB]))
# image2.show()
# image2.save('temp.png')
image2

In [ ]:
small_rois.shape
small_rois[iB, :]

In [ ]:
small_rois_centers = (small_rois[:, 0:2] + small_rois[:, 2:4])/2

In [ ]:
small_rois_centers.shape

In [ ]:
from sklearn.cluster import MeanShift
clustering = MeanShift(bandwidth=single_item_width).fit(small_rois_centers)
num_clusters = len(set(clustering.labels_))
mean_clusterred_rois = np.zeros((num_clusters, 4), dtype = "float32")
for iC in range(num_clusters):
    mean_clusterred_rois[iC] = np.mean(small_rois[np.where(clustering.labels_ == iC)[0], :], axis=0)
mean_clusterred_rois.shape

In [ ]:
image3 = copy(image)
draw3 = ImageDraw.Draw(image3)
for iB in mean_clusterred_rois:
    print(iB)
    draw3.rectangle(iB, outline = (200,200,200))

In [ ]:
image3

In [ ]:
print(b.shape)
print(c.shape)
print(d.shape)

In [ ]:
mean_clusterred_rois = np.concatenate((np.zeros((1, num_clusters)).T, mean_clusterred_rois), axis=1)

In [ ]:
mean_clusterred_rois

In [ ]:
def find_the_popout(X):
    X = mean_clusterred_rois
    num_items = X.shape[0]
    dists_matrix  = (np.sum(X**2, axis = 1) - 2 * X.dot(X.T)).T + np.sum(X**2, axis = 1)
    sum_dists_from_others = dists_matrix.sum(axis =0)
    popout_index = np.argmax(sum_dists_from_others)
    popout_rois = mean_clusterred_rois[popout_index, 1:5]
    return popout_rois

In [ ]:
# import torch
# def find_the_popout(X):
#     X = torch.from_numpy(mean_clusterred_rois)
#     num_items = X.shape[0]
# #     dists_matrix  = (torch.sum(X**2, dim= 1) - 2 * X.mm(X.t())).t() + torch.sum(X**2, dim= 1)
#     dists_matrix  = (torch.sum(X**2, dim= 1) - 2 * torch.mm(X, X.t())).t() + torch.sum(X**2, dim= 1)
#     sum_dists_from_others = torch.sum(dists_matrix, dim =0)
#     popout_index = torch.max(sum_dists_from_others)
#     popout_rois = mean_clusterred_rois[popout_index, 1:5]
#     return popout_rois

In [ ]:
my_feat = np.load('my_feat.npy')
popout_rois = find_the_popout(my_feat)

In [ ]:
image4 = copy(image)
draw4 = ImageDraw.Draw(image4)
draw4.rectangle(np.float32(popout_rois), outline = (200,200,200))
image4

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(dists_matrix)
plt.colorbar()
plt.show()

In [ ]:
X = my_feat
num_items = X.shape[0]
dists_matrix  = (np.sum(X**2, axis = 1) - 2 * X.dot(X.T)).T + np.sum(X**2, axis = 1)
sum_dists_from_others = dists_matrix.sum(axis =0)
popout_index = np.argmax(sum_dists_from_others)
popout_rois = X[popout_index, 1:5]


In [ ]:
popout_rois.dtype

In [ ]:
a =find_the_popout(mean_clusterred_rois)

In [ ]:
my_feat.shape

In [ ]:
small_rois[iB, :]

In [ ]:
feat_df = pd.DataFrame(my_feat.T, index = range(2048))
plt.figure() 
feat_df.plot(alpha=0.5)
plt.show()

In [ ]:
ts
df;

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list('ABCD'))
df = df.cumsum()

In [ ]:
plt.figure() 
df.plot()
plt.show()